### Library Imports

In [ ]:
# Reload Extensions
%load_ext autoreload
%autoreload 2

# Base Libraries
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from utility_scripts import utils
import os


# DL Base Libraries
import tensorflow as tf
import tensorflow.keras.applications as tf_app
from tensorflow.keras.applications.mobilenet_v3 import (
    preprocess_input,
    decode_predictions,
)
from EMA import (
    EMA_finalize,
    EMA_init,
)

# Use GPUS as is Required
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [ ]:
nb = int(input("\n\nIf using a jnb enter 1 else 0\n"))
uni = int(input("\n\nIf using uni gpu enter 1 else 0\n"))
gpu_id = int(
    input("\n\n Which GPU to USE enter the GPU index of nvidia-smi\n")
)
# If everything is run from jupyter nb then results file generated will have a suffix of uni

results_suffix = "uni" if uni else "work"
results_suffix += "_nb" if nb else "_py"
print(f"Suffix used with result files will be {results_suffix}!!")


GPU_ID = gpu_id
print(f"GPU: {GPU_ID} is being used")

### Downloading the Model

In [ ]:
# Define the model names and directories for saving
model_name = "MobileNetV3L"
models_directory = "models_lib"
results_directory = "benchmark_results"

model_type = "tf_models"
model_save_path = os.path.join(models_directory, model_type)
results_save_path = os.path.join(results_directory, model_name)

if not os.path.exists(model_save_path):
    print(f"Save Path {model_save_path} doesn't exist Creating!!")
    os.makedirs(model_save_path, exist_ok=True)


if not os.path.exists(results_save_path):
    print(f"Results Dir {results_save_path} doesn't exist Creating!!")
    os.makedirs(results_save_path, exist_ok=True)

url = "https://i.pinimg.com/originals/56/ea/2b/56ea2bb991a7446776ac2f2f27fdc397.jpg"
num_images = 512
# url = "https://images.dog.ceo/breeds/retriever-golden/n02099601_3004.jpg"

In [ ]:
# Loading the model
tf_model = tf_app.MobileNetV3Large(weights="imagenet", include_top=True)

In [ ]:
# Initialization.
# EMA_init()
img = resize(io.imread(url), (224, 224))
img = 255 * np.expand_dims(img, axis=0)
img = preprocess_input(img)
preds = tf_model.predict(img)
print(f"Predicted {decode_predictions(preds, top = 3)[0]}")
plt.imshow(img[0] / 255)
plt.title(decode_predictions(preds, top=3)[0][0][1])
plt.axis()
plt.show()

### Saving the Model

In [ ]:
# tf_model.save(os.path.join(model_save_path, model_name))
# tf_model.save(f"{os.path.join(model_save_path, model_name)}.keras")

In [ ]:
tf_model = tf.keras.models.load_model(
    os.path.join(model_save_path, model_name)
)
keras_model = tf.keras.models.load_model(
    f"{os.path.join(model_save_path, model_name)}.keras"
)

#### Making some preds

In [ ]:
# Loading n preprocessing the image

img = resize(io.imread(url), (224, 224))
img = 255 * np.expand_dims(img, axis=0)
img = preprocess_input(img)
preds = keras_model(img).numpy()
print(f"Predicted {decode_predictions(preds, top = 3)[0]}")
plt.imshow(img[0] / 255)
plt.title(decode_predictions(preds, top=3)[0][0][1])
plt.axis()
plt.show()


xla_enabled_model = tf.function(keras_model)
tf.config.optimizer.set_jit(True)
preds = xla_enabled_model(img).numpy()
print(f"Predicted {decode_predictions(preds, top = 3)[0]}")
plt.imshow(img[0] / 255)
plt.title(decode_predictions(preds, top=3)[0][0][1])
plt.axis()
plt.show()

In [ ]:
### Doing one prediction is necessary to compile the model
# Loading n preprocessing the image

img = resize(io.imread(url), (224, 224))
img = 255 * np.expand_dims(img, axis=0)
img = preprocess_input(img)
preds = tf_model(img).numpy()
print(f"Predicted {decode_predictions(preds, top = 3)[0]}")
plt.imshow(img[0] / 255)
plt.title(decode_predictions(preds, top=3)[0][0][1])
plt.axis()
plt.show()


xla_enabled_model = tf.function(tf_model)
tf.config.optimizer.set_jit(True)
preds = xla_enabled_model(img).numpy()
print(f"Predicted {decode_predictions(preds, top = 3)[0]}")
plt.imshow(img[0] / 255)
plt.title(decode_predictions(preds, top=3)[0][0][1])
plt.axis()
plt.show()

### Tensorflow H5 Model Performance Measurement

In [ ]:
### Initialization of EMA
EMA_init()

In [ ]:
img = io.imread(url)
input_data, _ = utils.batch_sigle_img(
    img,
    target_size=(224, 224),
    num_images=num_images,
    preprocessor=preprocess_input,
)
input_data = input_data.astype(np.float32)
num_warmup_runs = 50
num_model_runs = 10

In [ ]:
fname = f"tf_{num_model_runs}_it_{results_suffix}.csv"
csv_save_path = os.path.join(results_save_path, fname)

utils.batch_model_performances(
    framework_name="tf",
    model=tf_model,
    input_data=input_data,
    batch_sizes=[8, 16, 32, 64, 128, 256, 512],
    csv_path=csv_save_path,
    num_warmup_runs=num_warmup_runs,
    num_model_runs=num_model_runs,
    trt=False,
    onnx=False,
    torch=False,
    gpu_id=GPU_ID,
)

In [ ]:
# NOTE Remember to export xargs
fname = f"tfxla_{num_model_runs}_it_{results_suffix}.csv"
csv_save_path = os.path.join(results_save_path, fname)

xla_enabled_model = tf.function(tf_model)
tf.config.optimizer.set_jit(True)
utils.batch_model_performances(
    framework_name="tfxla",
    model=xla_enabled_model,
    input_data=input_data,
    batch_sizes=[8, 16, 32, 64, 128, 256, 512],
    csv_path=csv_save_path,
    num_warmup_runs=num_warmup_runs,
    num_model_runs=num_model_runs,
    trt=False,
    onnx=False,
    torch=False,
    gpu_id=GPU_ID,
)

In [ ]:
fname = f"keras_{num_model_runs}_it_{results_suffix}.csv"
csv_save_path = os.path.join(results_save_path, fname)
utils.batch_model_performances(
    framework_name="keras",
    model=keras_model,
    input_data=input_data,
    batch_sizes=[8, 16, 32, 64, 128, 256, 512],
    csv_path=csv_save_path,
    num_warmup_runs=num_warmup_runs,
    num_model_runs=num_model_runs,
    trt=False,
    onnx=False,
    torch=False,
    gpu_id=GPU_ID,
)

In [ ]:
# NOTE Remember to export xargs
fname = f"kerasxla_{num_model_runs}_it_{results_suffix}.csv"
csv_save_path = os.path.join(results_save_path, fname)

xla_enabled_model = tf.function(keras_model)
tf.config.optimizer.set_jit(True)
utils.batch_model_performances(
    framework_name="kerasxla",
    model=xla_enabled_model,
    input_data=input_data,
    batch_sizes=[8, 16, 32, 64, 128, 256, 512],
    csv_path=csv_save_path,
    num_warmup_runs=num_warmup_runs,
    num_model_runs=num_model_runs,
    trt=False,
    onnx=False,
    torch=False,
    gpu_id=GPU_ID,
)

In [ ]:
# Finishing the EMA
EMA_finalize()